In [27]:
import os
import json
import statistics

directory = './data/detailed_filtered_data'
threshold = 370

print("[ Median | AVG ] number of posts left per user after filtering all posts shorter than 370 chars: ")
for root, dirs, files in os.walk(directory):
    for folder in dirs:
        counter_in_genre = []
        folder_path = os.path.join(root, folder)
        for file in os.listdir(folder_path):
            if file.endswith('.json'):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    counter = 0
                    for community in data.keys():
                        for sub in data.get(community, {}).get('submissions'):
                            if len(sub.get("title", "")) >= threshold:
                                counter += 1
                            if len(sub.get("body", "")) >= threshold:
                                counter += 1
                        for comment in data.get(community, {}).get('comments'):
                            if len(comment.get("body", "")) >= threshold:
                                counter += 1
                    counter_in_genre.append(counter)
        if counter_in_genre:
            print(folder, "[", statistics.median(counter_in_genre),
                  "|", statistics.mean(counter_in_genre), "]")

[ Median | AVG ] number of posts left per user after filtering all posts shorter than 370 chars: 
indieheads [ 13.0 | 41.102 ]
hiphopheads [ 28.0 | 54.697 ]
Metal [ 21.0 | 51.493 ]
electronicmusic [ 17.0 | 48.915 ]
classicalmusic [ 59.0 | 101.091 ]
